## Calculating Codon Adaptation Index according (CAI) according to Sharp et al. 1987

The codon adaptation index (CAI) measures the level of adaptation of a gene to a certain organism. 

It was first introduced by [Sharp 1987](https://www.ncbi.nlm.nih.gov/pubmed/3547335). CAI measures the deviation of the codons in a given protein coding gene sequence with respect to the codons used in a reference set of genes. See the [wikipedia](https://en.wikipedia.org/wiki/Codon_Adaptation_Index) article on CAI.

There are at least two python implementations for calculating CAI.

[Biopython](http://biopython.org) has a module called [CodonUsage](http://biopython.org/DIST/docs/api/Bio.SeqUtils.CodonUsage.CodonAdaptationIndex-class.html). This module does not contain any of the necessary reference data in order to calculate CAI for yeast. 
Worse, the fidelity of the Biopython module had been questioned in this exchange on [Biostars](https://www.biostars.org/p/290485/).

The python module [CAI](https://pypi.org/project/CAI/) is an alternative implementation.

The purpose of this notebook is to test the fidelity of both the Biopython and CAI modules using the original data from Sharp 1987.

Codon adaptation data was tabulated in Table 1 [Sharp 1987](https://www.ncbi.nlm.nih.gov/pubmed/3547335).

![Sharp Table 1](sharp_table_1.png)

The Sharp Table 1 was copied and edited from the pdf version of the publication. 

The order of the columns remains the same.

RSCU and weights (w) are tabulated for 61 of the 64 codons (missing are the stop codons TAA, TAG and TGA).






In [1]:
sharp_table_1 = """
AA Tri RSCUe we RSCUy wy
Phe TTT 0.456 0.296 0.203 0.113
Phe TTC 1.544 1.000 1.797 1.000
Leu TTA 0.106 0.020 0.601 0.117
Leu TTG 0.106 0.020 5.141 1.000
Leu CTT 0.225 0.042 0.029 0.006
Leu CTC 0.198 0.037 0.014 0.003
Leu CTA 0.040 0.007 0.200 0.039
Leu CTG 5.326 1.000 0.014 0.003
Ile ATT 0.466 0.185 1.352 0.823
Ile ATC 2.525 1.000 1.643 1.000
Ile ATA 0.008 0.003 0.005 0.003
Met ATG 1.000 1.000 1.000 1.000
Val GTT 2.244 1.000 2.161 1.000
Val GTC 0.148 0.066 1.796 0.831
Val GTA 1.111 0.495 0.004 0.002
Val GTG 0.496 0.221 0.039 0.018
Ser TCT 2.571 1.000 3.359 1.000
Ser TCC 1.912 0.744 2.327 0.693
Ser TCA 0.198 0.077 0.122 0.036
Ser TCG 0.044 0.017 0.017 0.005
Pro CCT 0.231 0.070 0.179 0.047
Pro CCC 0.038 0.012 0.036 0.009
Pro CCA 0.442 0.135 3.776 1.000
Pro CCG 3.288 1.000 0.009 0.002
Thr ACT 1.804 0.965 1.899 0.921
Thr ACC 1.870 1.000 2.063 1.000
Thr ACA 0.141 0.076 0.025 0.012
Thr ACG 0.185 0.099 0.013 0.006
Ala GCT 1.877 1.000 3.005 1.000
Ala GCC 0.228 0.122 0.948 0.316
Ala GCA 1.099 0.586 0.044 0.015
Ala GCG 0.796 0.424 0.004 0.001
Tyr TAT 0.386 0.239 0.132 0.071
Tyr TAC 1.614 1.000 1.868 1.000
His CAT 0.451 0.291 0.394 0.245
His CAC 1.549 1.000 1.606 1.000
Gln CAA 0.220 0.124 1.987 1.000
Gln CAG 1.780 1.000 0.013 0.007
Asn AAT 0.097 0.051 0.100 0.053
Asn AAC 1.903 1.000 1.900 1.000
Lys AAA 1.596 1.000 0.237 0.135
Lys AAG 0.404 0.253 1.763 1.000
Asp GAT 0.605 0.434 0.713 0.554
Asp GAC 1.395 1.000 1.287 1.000
Glu GAA 1.589 1.000 1.968 1.000
Glu GAG 0.411 0.259 0.032 0.016
Cys TGT 0.667 0.500 1.857 1.000
Cys TGC 1.333 1.000 0.143 0.077
Trp TGG 1.000 1.000 1.000 1.000
Arg CGT 4.380 1.000 0.718 0.137
Arg CGC 1.561 0.356 0.008 0.002
Arg CGA 0.017 0.004 0.008 0.002
Arg CGG 0.017 0.004 0.008 0.002
Ser AGT 0.220 0.085 0.070 0.021
Ser AGC 1.055 0.410 0.105 0.031
Arg AGA 0.017 0.004 5.241 1.000
Arg AGG 0.008 0.002 0.017 0.003
Gly GGT 2.283 1.000 3.898 1.000
Gly GGC 1.652 0.724 0.077 0.020
Gly GGA 0.022 0.010 0.009 0.002
Gly GGG 0.043 0.019 0.017 0.004"""

Pandas can turn this text table into a dataframe

In [2]:
import pandas as pd
from io import StringIO

In [3]:
sharp_df = pd.read_csv(StringIO(sharp_table_1.strip()), sep=" ")

The resulting dataframe seem to reflect the original table above.

In [79]:
sharp_df

,AA,Tri,RSCUe,we,RSCUy,wy
0,Phe,TTT,0.456,0.296,0.203,0.113
1,Phe,TTC,1.544,1.000,1.797,1.000
2,Leu,TTA,0.106,0.020,0.601,0.117
3,Leu,TTG,0.106,0.020,5.141,1.000
4,Leu,CTT,0.225,0.042,0.029,0.006
...,...,...,...,...,...,...
56,Arg,AGG,0.008,0.002,0.017,0.003
57,Gly,GGT,2.283,1.000,3.898,1.000
58,Gly,GGC,1.652,0.724,0.077,0.020
59,Gly,GGA,0.022,0.010,0.009,0.002


Each row is combined into a dict

In [64]:
RSCUe = dict(zip(sharp_df["Tri"],round(sharp_df["RSCUe"],3)))
we = dict(zip(sharp_df["Tri"],round(sharp_df["we"],3)))
RSCUy = dict(zip(sharp_df["Tri"],round(sharp_df["RSCUy"],3)))
wy = dict(zip(sharp_df["Tri"],round(sharp_df["wy"],3)))

In [68]:
RSCUe["TTT"]

0.456

In [69]:
we["TTT"]

0.296

The `we` dict should be the same as the `SharpEcoliIndex` from biopython

In [70]:
from Bio.SeqUtils.CodonUsageIndices import SharpEcoliIndex

This is actually so:

In [71]:
SharpEcoliIndex == we

True

Some examples of CAI values calculated from genes 
for _E. coli_ and _S. cerevisiae_ genes 
are given in table 2 of Sharp 1987. 


[![Sharp Table 2](sharp_table_2.png)](sharp_table_2.png)


The table was transferred to text format:


| E.coli 	|             	| yeast      	|             	|
|--------	|-------------	|------------	|-------------	|
| gene   	| CAI         	| gene       	| CAI         	|
| 17 RPs 	| 0.467-0.813 	| 16 RPs     	| 0.529-0.915 	|
| rpsU   	| 0.726       	| histones   	| 0.529-0.915 	|
| rpoD   	| 0.582       	| 2u plasmid 	| 0.099-0.106 	|
| dnaG   	| 0.271       	| GAL4       	| 0.116       	|
| lacI   	| 0.296       	| PPR1       	| 0.114       	|
| trpR   	| 0.267       	| GPD1      	| 0.929       	|
| lpp    	| 0.849       	| matA2      	| 0.098       	|
| hsdS   	| 0.218       	|            	|             	|
|        	|             	|            	|             	|
|        	|             	|            	|             	|


The rpsU gene i given as an example on page 1285. The sequence is available from NC_000913 REGION: 3210781..3210996.

    atg ccg gta att aaa gta cgtgaaaacgagccgttcgacgtagctctgcgtcgctt
       .CCG.GTA.ATT.AAA.GTA.
       ---===---===---
    caagcgttcctgcgaaaaagcaggtgttctggcggaagttcgtcgtcgtgagttctatgaaaaaccgactaccgaacgtaagcg
    cgctaaagcttctgcagtgaaacgtcacgcgaagaaactggctcgcgaaaacgcacgccgcactcgtctgtactaa

In [9]:
from pydna.genbank import genbank

In [10]:
rpsU = str(genbank("NC_000913 REGION: 3210781..3210996").seq)
print(rpsU)

ATGCCGGTAATTAAAGTACGTGAAAACGAGCCGTTCGACGTAGCTCTGCGTCGCTTCAAGCGTTCCTGCGAAAAAGCAGGTGTTCTGGCGGAAGTTCGTCGTCGTGAGTTCTATGAAAAACCGACTACCGAACGTAAGCGCGCTAAAGCTTCTGCAGTGAAACGTCACGCGAAGAAACTGGCTCGCGAAAACGCACGCCGCACTCGTCTGTACTAA


The data can be used to test the prediction of the biopython module.

In [72]:
from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex

In [73]:
cai = CodonAdaptationIndex()

In [74]:
cai.set_cai_index(we)

In [75]:
round(cai.cai_for_gene(rpsU), 3)

0.723

The biopython module show a small difference, 0.723 instead of 0.726

The CAI module calculates the same value for rpsU as given in the paper.

from CAI import CAI

In [77]:
round(CAI(rpsU, weights=we),3)

0.726

In [ ]:
The CAI module can also accept 

In [17]:
from pygenome import saccharomyces_cerevisiae as sg

In [18]:
GAL4 = str(sg.stdgenes["GAL4"].cds().seq)
PPR1 = str(sg.stdgenes["PPR1"].cds().seq)
GPD1 = str(sg.stdgenes["TDH3"].cds().seq)

In [19]:
cai.set_cai_index(wy)

In [20]:
print(round(cai.cai_for_gene(GAL4),3))
print(round(cai.cai_for_gene(PPR1),3))
print(round(cai.cai_for_gene(GPD1),3))

0.116
0.114
0.924


In [21]:
print(round(CAI(GAL4, weights=wy),3))
print(round(CAI(PPR1, weights=wy),3)) 
print(round(CAI(GPD1, weights=wy),3)) 

0.116
0.115
0.924


There was a small differences for PPR1 (<1% for the CAI module) and a lower value for GDP/TDH3 (0.5%), this could perhaps be explained by rounding.

Both implementations seem mathematically correct.

Some companies such as [Genscript]()
offer servieces to optimize a gene for a specific host (for example [here](https://www.genscript.com/tools/rare-codon-analysis)). A part of this analysis involves CAI.

In [22]:
print(GPD1)

ATGGTTAGAGTTGCTATTAACGGTTTCGGTAGAATCGGTAGATTGGTCATGAGAATTGCTTTGTCTAGACCAAACGTCGAAGTTGTTGCTTTGAACGACCCATTCATCACCAACGACTACGCTGCTTACATGTTCAAGTACGACTCCACTCACGGTAGATACGCTGGTGAAGTTTCCCACGATGACAAGCACATCATTGTCGATGGTAAGAAGATTGCTACTTACCAAGAAAGAGACCCAGCTAACTTGCCATGGGGTTCTTCCAACGTTGACATCGCCATTGACTCCACTGGTGTTTTCAAGGAATTAGACACTGCTCAAAAGCACATTGACGCTGGTGCCAAGAAGGTTGTTATCACTGCTCCATCTTCCACCGCCCCAATGTTCGTCATGGGTGTTAACGAAGAAAAATACACTTCTGACTTGAAGATTGTTTCCAACGCTTCTTGTACCACCAACTGTTTGGCTCCATTGGCCAAGGTTATCAACGATGCTTTCGGTATTGAAGAAGGTTTGATGACCACTGTCCACTCTTTGACTGCTACTCAAAAGACTGTTGACGGTCCATCCCACAAGGACTGGAGAGGTGGTAGAACCGCTTCCGGTAACATCATCCCATCCTCCACCGGTGCTGCTAAGGCTGTCGGTAAGGTCTTGCCAGAATTGCAAGGTAAGTTGACCGGTATGGCTTTCAGAGTCCCAACCGTCGATGTCTCCGTTGTTGACTTGACTGTCAAGTTGAACAAGGAAACCACCTACGATGAAATCAAGAAGGTTGTTAAGGCTGCCGCTGAAGGTAAGTTGAAGGGTGTTTTGGGTTACACCGAAGACGCTGTTGTCTCCTCTGACTTCTTGGGTGACTCTCACTCTTCCATCTTCGATGCTTCCGCTGGTATCCAATTGTCTCCAAAGTTCGTCAAGTTGGTCTCCTGGTACGACAACGAATACGGTTACTCTACCAGAGTTGTCGACTTGGTTGAACACGTTGCCAAGGCTTAA


The Genscript website gave very different values (2021-03-17) for the genes, the values are spaced more closely.

| gene  	| Sharp 	| Genscript 	|
|-------	|-------	|-----------	|
| GAL4  	| 0.116 	| 0.71      	|
| PPR1  	| 0.114 	| 0.73      	|
| GPD1  	| 0.929 	| 0.82      	|

The cause for this could be the use of a different reference dataset. Many datasets for many organisms can be found at the [codon usage database](https://www.kazusa.or.jp/codon). This database uses a text format called "Kazusa". 

CAI reference data for S. cerevisiae can be found [here](https://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=4932)

The setuptools package [python-codon-tables](https://pypi.org/project/python-codon-tables/) should be able to access this data directly:

In [23]:
# Remove # below to run pip in jupyter notebook
#     %pip install python-codon-tables

In [27]:
import python_codon_tables as pct

In [29]:
pct.available_codon_tables_names

['g_gallus_9031',
 'b_subtilis_1423',
 'd_melanogaster_7227',
 'm_musculus_domesticus_10092',
 's_cerevisiae_4932',
 'c_elegans_6239',
 'h_sapiens_9606',
 'm_musculus_10090',
 'e_coli_316407']

In [30]:
table = pct.get_codons_table('s_cerevisiae_4932')

In [39]:
table

{'*': {'TAA': 0.47, 'TAG': 0.23, 'TGA': 0.3},
 'A': {'GCA': 0.29, 'GCC': 0.22, 'GCG': 0.11, 'GCT': 0.38},
 'C': {'TGC': 0.37, 'TGT': 0.63},
 'D': {'GAC': 0.35, 'GAT': 0.65},
 'E': {'GAA': 0.7, 'GAG': 0.3},
 'F': {'TTC': 0.41, 'TTT': 0.59},
 'G': {'GGA': 0.22, 'GGC': 0.19, 'GGG': 0.12, 'GGT': 0.47},
 'H': {'CAC': 0.36, 'CAT': 0.64},
 'I': {'ATA': 0.27, 'ATC': 0.26, 'ATT': 0.46},
 'K': {'AAA': 0.58, 'AAG': 0.42},
 'L': {'CTA': 0.14,
  'CTC': 0.06,
  'CTG': 0.11,
  'CTT': 0.13,
  'TTA': 0.28,
  'TTG': 0.29},
 'M': {'ATG': 1.0},
 'N': {'AAC': 0.41, 'AAT': 0.59},
 'P': {'CCA': 0.42, 'CCC': 0.15, 'CCG': 0.12, 'CCT': 0.31},
 'Q': {'CAA': 0.69, 'CAG': 0.31},
 'R': {'AGA': 0.48,
  'AGG': 0.21,
  'CGA': 0.07,
  'CGC': 0.06,
  'CGG': 0.04,
  'CGT': 0.14},
 'S': {'AGC': 0.11,
  'AGT': 0.16,
  'TCA': 0.21,
  'TCC': 0.16,
  'TCG': 0.1,
  'TCT': 0.26},
 'T': {'ACA': 0.3, 'ACC': 0.22, 'ACG': 0.14, 'ACT': 0.35},
 'V': {'GTA': 0.21, 'GTC': 0.21, 'GTG': 0.19, 'GTT': 0.39},
 'W': {'TGG': 1.0},
 'Y': {'TAC

this format does not seem to be directly compatible with CAI or biopython modules. We can fix this:

In [47]:
kazusaw = {}
for dct in table.values():
    for trip, w in dct.items():
        kazusaw[trip] = w 

The resulting dict weights has the correct format for the CAI and biopython modules. 

In [48]:
print(kazusaw)

{'TAA': 0.47, 'TAG': 0.23, 'TGA': 0.3, 'GCA': 0.29, 'GCC': 0.22, 'GCG': 0.11, 'GCT': 0.38, 'TGC': 0.37, 'TGT': 0.63, 'GAC': 0.35, 'GAT': 0.65, 'GAA': 0.7, 'GAG': 0.3, 'TTC': 0.41, 'TTT': 0.59, 'GGA': 0.22, 'GGC': 0.19, 'GGG': 0.12, 'GGT': 0.47, 'CAC': 0.36, 'CAT': 0.64, 'ATA': 0.27, 'ATC': 0.26, 'ATT': 0.46, 'AAA': 0.58, 'AAG': 0.42, 'CTA': 0.14, 'CTC': 0.06, 'CTG': 0.11, 'CTT': 0.13, 'TTA': 0.28, 'TTG': 0.29, 'ATG': 1.0, 'AAC': 0.41, 'AAT': 0.59, 'CCA': 0.42, 'CCC': 0.15, 'CCG': 0.12, 'CCT': 0.31, 'CAA': 0.69, 'CAG': 0.31, 'AGA': 0.48, 'AGG': 0.21, 'CGA': 0.07, 'CGC': 0.06, 'CGG': 0.04, 'CGT': 0.14, 'AGC': 0.11, 'AGT': 0.16, 'TCA': 0.21, 'TCC': 0.16, 'TCG': 0.1, 'TCT': 0.26, 'ACA': 0.3, 'ACC': 0.22, 'ACG': 0.14, 'ACT': 0.35, 'GTA': 0.21, 'GTC': 0.21, 'GTG': 0.19, 'GTT': 0.39, 'TGG': 1.0, 'TAC': 0.44, 'TAT': 0.56}


In [49]:
cai.set_cai_index(kazusaw)

In [53]:
print(round(cai.cai_for_gene(GAL4),3))
print(round(cai.cai_for_gene(PPR1),3))
print(round(cai.cai_for_gene(GPD1),3))

0.31
0.327
0.362


In [54]:
print(round(CAI(GAL4, weights=kazusaw),3))
print(round(CAI(PPR1, weights=kazusaw),3)) 
print(round(CAI(GPD1, weights=kazusaw),3)) 

0.311
0.327
0.363


The results do not compare well.

This service 

[CAIcal SERVER](http://genomes.urv.es/CAIcal)

https://wiki.yeastgenome.org/index.php/S._cerevisiae_Codon_Usage_Tables


    Saccharomyces cerevisiae [gbpln]: 14411 CDS's (6534504 codons)
    fields: [triplet] [frequency: per thousand] ([number])
    UUU 26.1(170666)  UCU 23.5(153557)  UAU 18.8(122728)  UGU  8.1( 52903)
    UUC 18.4(120510)  UCC 14.2( 92923)  UAC 14.8( 96596)  UGC  4.8( 31095)
    UUA 26.2(170884)  UCA 18.7(122028)  UAA  1.1(  6913)  UGA  0.7(  4447)
    UUG 27.2(177573)  UCG  8.6( 55951)  UAG  0.5(  3312)  UGG 10.4( 67789)

    CUU 12.3( 80076)  CCU 13.5( 88263)  CAU 13.6( 89007)  CGU  6.4( 41791)
    CUC  5.4( 35545)  CCC  6.8( 44309)  CAC  7.8( 50785)  CGC  2.6( 16993)
    CUA 13.4( 87619)  CCA 18.3(119641)  CAA 27.3(178251)  CGA  3.0( 19562)
    CUG 10.5( 68494)  CCG  5.3( 34597)  CAG 12.1( 79121)  CGG  1.7( 11351)

    AUU 30.1(196893)  ACU 20.3(132522)  AAU 35.7(233124)  AGU 14.2( 92466)
    AUC 17.2(112176)  ACC 12.7( 83207)  AAC 24.8(162199)  AGC  9.8( 63726)
    AUA 17.8(116254)  ACA 17.8(116084)  AAA 41.9(273618)  AGA 21.3(139081)
    AUG 20.9(136805)  ACG  8.0( 52045)  AAG 30.8(201361)  AGG  9.2( 60289)

    GUU 22.1(144243)  GCU 21.2(138358)  GAU 37.6(245641)  GGU 23.9(156109)
    GUC 11.8( 76947)  GCC 12.6( 82357)  GAC 20.2(132048)  GGC  9.8( 63903)
    GUA 11.8( 76927)  GCA 16.2(105910)  GAA 45.6(297944)  GGA 10.9( 71216)
    GUG 10.8( 70337)  GCG  6.2( 40358)  GAG 19.2(125717)  GGG  6.0( 39359)



![](GAL4b.png)
![](PPR1.png)
![](TDH3.png)


| gene  	| Sharp 	| Genscript 	| CAIcal |
|-------	|-------	|-----------	| ------ |
| GAL4  	| 0.116 	| 0.71      	| 0.761  |
| PPR1  	| 0.114 	| 0.73      	| 0.770  |
| GPD1  	| 0.929 	| 0.82      	| 0.813  |